# Liquid-only thermometry
- This script shows how to use the various options for liquid-only thermometry
- This tutorial is also a great starting point when using  other thermobarometers, as the choice of inputs is the same as for other phases
- You can download the Excel spreadsheet you need here: https://github.com/PennyWieser/Thermobar/blob/main/docs/Examples/Liquid_Ol_Liq_Themometry/Liquid_only_Thermometry.xlsx

### You need to install Thermobar once on your machine, if you haven't done this yet, uncomment the line below (remove the #)

In [1]:
#!pip install Thermobar

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Thermobar as pt

In [4]:
pt.__version__

'1.0.15'

## Load in data
- The function "import_excel" allows you to specify the name of the excel file and the sheet name.
- Data should have the headings SiO2_Liq, TiO2_Liq etc. etc. The order of headings doesn't matter.
- As in the 5 min intro example, you could also have no _Liq in the headings, and load them in the functoin using suffix="_Liq"
- You can also have any other columns, e.g., estimate of pressure from any other proxy (melt inclusions, geophysics), and anything else you might want to plot (e.g., latitude, longitude)

In [5]:
out=pt.import_excel('2021_LP_MI_Olivine_MI_PEC_Glasses.xlsx', sheet_name="Liq")
my_input=out['my_input']
myLiquids1=out['Liqs']

### Inspect inputs to check you didn't have any funny column headings.
- It is always useful to inspect the outputs from import_excel, sometimes your column headings may have funny characters due to use of spaces, subscripts etc. in journal pdf tables. Check that all the columns you entered have numbers. If, say your SiO2_Liq heading had funny characters, this column will be filled with zeros when you inspect it.

In [6]:
myLiquids1.head()

,SiO2_Liq,TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,CaO_Liq,Na2O_Liq,K2O_Liq,Cr2O3_Liq,P2O5_Liq,H2O_Liq,Fe3Fet_Liq,NiO_Liq,CoO_Liq,CO2_Liq,Sample_ID_Liq
0,43.840,3.876,14.643,12.134,0.149,6.802,10.437,3.568,1.539,0.000,0.767,2.019761,0.3,0.0,0.0,0.0,0
1,45.059,3.695,14.797,12.137,0.131,6.830,10.245,3.330,1.939,0.002,0.078,1.570071,0.3,0.0,0.0,0.0,1
2,43.051,4.015,14.360,12.200,0.139,6.394,11.479,3.485,1.383,0.000,0.981,2.236740,0.3,0.0,0.0,0.0,2
3,42.657,4.024,14.509,12.291,0.158,6.457,11.453,3.437,1.315,0.000,0.985,2.439778,0.3,0.0,0.0,0.0,3
4,42.614,4.009,14.557,12.193,0.165,6.380,11.450,3.607,1.376,0.000,1.023,2.341731,0.3,0.0,0.0,0.0,4


## If at any point you need help with what functions inputs to use, you can type help(pt.function_name)

In [7]:
# Options for different thermometers laid out in the "equationT" =
help(pt.calculate_liq_only_temp)

Help on function calculate_liq_only_temp in module Thermobar.liquid_thermometers:

calculate_liq_only_temp(*, liq_comps, equationT, P=None, H2O_Liq=None, print=False)
     Liquid-only thermometery. Returns a temperature in Kelvin.
    
    Parameters
     -------
    
     liq_comps: pandas.DataFrame
         liquid compositions with column headings SiO2_Liq, MgO_Liq etc.
    
     equationT: str
         If has _sat at the end, represents the saturation surface of that mineral.
    
         Equations from Putirka et al. (2016).
             | T_Put2016_eq3_amp_sat (saturation surface of amphibole)
    
         Equations from Putirka (2008) and older studies:
    
             | T_Put2008_eq13
             | T_Put2008_eq14
             | T_Put2008_eq15
             | T_Put2008_eq16
             | T_Put2008_eq34_cpx_sat
             | T_Put2008_eq28b_opx_sat
             | T_Put1999_cpx_sat
             * Following 3 thermometers are adaptations of olivine-liquid thermometers with  DM

## Example 1 - P and H2O-independent thermometers
- The thermometer of Helz and Thornber 1987 is not dependent on pressure or H2O.
- Thus, the function only requires 2 inputs, the name of pandas dataframe with liquid compositions (here, myLiquids1) and the name of the equationT you have choosen
- All functions output in Kelvin and Kbar, so to get celcius, we subtract 273.15

In [8]:
T_Helz1987=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Helz1987_MgO")-273.15
# In jupyter lab/notebooks, outputs don't automatically display, 
# but it prints the last line in each cell
T_Helz1987

0     1150.7202
1     1151.2830
2     1142.5194
3     1143.7857
4     1142.2380
        ...    
67    1152.5895
68    1140.0471
69    1127.4645
70    1134.0975
71    1134.7608
Name: MgO_Liq, Length: 72, dtype: float64

## Example 2 - Pressure-dependent thermometer 
a) If you select an equation which is Pressure-dependent, and don't specify a pressure, the code returns an error

b) You can either select P="Solve" which returns a partial function. This means you can evaluate it at any pressure you want easily

c) Or you can specify a fixed pressure, e.g., P=5 (which runs all calculations at 5 kbar)

d) Or you can specify pressure based on a column in your original spreadsheet containing pressure

## Example 2a - Don't specify a pressure
- returns an error, telling you to input a pressure

In [9]:
Teq15_2kbar_6wt=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
                equationT="T_Put2008_eq15")-273.15 # Convert to Celcius
Teq15_2kbar_6wt

ValueError: T_Put2008_eq15 requires you to enter P, or specify P="Solve"

## Example 2b - specify P="Solve" to return a partial function

In [10]:
Teq15_partial=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
            equationT="T_Put2008_eq15", P="Solve")# Output is in Kelvin

In [11]:
# You can then evaluate this partial at any pressure you want (here, 10 kbar).
# This can save computation time
Teq15_10kbar=Teq15_partial(10)-273.15
Teq15_10kbar

0     1204.207733
1     1211.766357
2     1189.668755
3     1188.202024
4     1188.703299
         ...     
67    1211.598076
68    1197.918547
69    1179.325179
70    1190.796133
71    1196.800410
Length: 72, dtype: float64

## Example 2c - evaluate at constant pressure

In [12]:
# Here P=5 kbar
Teq15_5kbar=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
            equationT="T_Put2008_eq15", P=6)-273.15 # Output is in Kelvin, convert to C
Teq15_5kbar 

0     1188.543733
1     1196.102357
2     1174.004755
3     1172.538024
4     1173.039299
         ...     
67    1195.934076
68    1182.254547
69    1163.661179
70    1175.132133
71    1181.136410
Length: 72, dtype: float64

## Example 2d - Evaluate at values given by P_kbar column in inputted excel spreadsheet
- your column can be called anything, you just need state P=my_input['column name']. If you stored pressure in GPa say, you could do P=10*my_input['P_GPa']

In [13]:
Teq15_input=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
            equationT="T_Put2008_eq15", P=my_input['P_kbar'])-273.15 
# Output is in Kelvin, convert to c.
Teq15_input

0     1188.543733
1     1196.102357
2     1174.004755
3     1172.538024
4     1173.039299
         ...     
67    1196.325676
68    1182.646147
69    1164.052779
70    1175.523733
71    1181.528010
Length: 72, dtype: float64

## Example 3 - Specifying water content
- The dataframe for liquids in this example has a H2O column it read from the excel spreadsheet. By default, H2O-dependent thermometers will use the values entered in this column
- If you don't have this column, it'll be filled with zeros, so calculations will be done at H2O=0
- However, you can also override this by specifying H2O_Liq=value in the function.

In [14]:
Teq15_2H2O=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
        equationT="T_Put2008_eq22_BeattDMg", P=my_input['P_kbar'], H2O_Liq=my_input['H2O_Liq'])-273.15 # Output is in Kelvin
Teq15_2H2O

0     1182.682119
1     1190.162090
2     1166.899401
3     1165.018830
4     1165.895935
         ...     
67    1192.209122
68    1178.621911
69    1158.984208
70    1170.779505
71    1178.084149
Length: 72, dtype: float64

In [15]:
Teq15_2H2O.to_csv('ALL LP MI Liq Putirka 2008 plus BeattDMg in C.csv') 

#### We can also investigate how sensitive our calculations are to water, say we think maybe water ranges from 2-4 wt%, we can do the same calculation at 4 wt%

In [38]:
Teq15_4H2O=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
                                      equationT="T_Put2008_eq15", P=5, H2O_Liq=4)-273.15 # Output is in Kelvin
Teq15_4H2O

0     1153.719755
1     1147.732697
2     1146.412219
3     1155.216428
4     1140.962939
5     1148.161127
6     1139.090296
7     1150.684424
8     1144.406584
9     1140.794102
10    1146.575601
11    1148.458646
12    1149.077266
13    1150.980620
14    1151.699878
15    1147.994865
16    1130.260588
17    1130.800903
18    1155.355256
19    1146.901892
20    1142.629000
21    1152.095323
22    1155.459468
23    1156.600608
24    1134.646142
25    1140.112690
26    1150.132571
27    1147.816788
28    1150.335374
29    1143.212923
30    1159.309139
31    1160.284850
32    1166.116595
33    1161.962557
34    1158.569179
35    1140.863637
36    1169.134793
37    1160.602277
38    1164.683315
39    1159.052426
40    1138.875366
41    1164.085671
42    1145.209682
43    1168.907706
44    1153.392108
45    1193.936388
46    1174.683699
47    1135.610638
48    1134.488773
49    1139.501749
50    1146.888356
51    1160.945060
52    1159.999192
53    1151.046353
54    1129.293709
55    1139

#### We can now calculate the difference between these by subtracting one from the other. Here, as H2O_Liq is just a constant on the function, the difference is constant for all liquids

In [39]:
Teq15_4H2O-Teq15_2H2O

0    -25.598751
1    -21.152013
2    -19.569803
3    -23.712227
4    -29.059312
5    -30.158139
6    -29.480518
7    -26.846804
8    -24.358843
9    -26.207965
10   -20.294551
11   -21.910654
12   -21.770068
13   -21.958291
14   -22.532339
15   -22.196344
16   -24.447813
17   -24.275449
18   -23.587341
19   -27.304416
20   -31.575431
21   -27.594324
22   -30.314462
23   -31.858015
24   -25.898586
25   -22.117851
26   -28.926785
27   -26.155795
28   -23.531847
29   -24.400138
30   -37.232276
31   -40.402167
32   -38.661981
33   -40.229204
34   -42.689020
35   -34.125849
36   -39.363144
37   -40.262075
38   -43.717249
39   -42.563459
40   -41.798666
41   -51.067487
42   -30.276990
43   -38.274279
44   -37.665225
45   -43.212241
46   -37.690013
47   -31.397101
48   -34.215201
49   -37.816560
50   -39.708410
51   -38.250410
52   -34.030941
53   -34.302842
54   -30.294020
55   -31.897631
56   -38.987065
dtype: float64

## Exporting data
- There are a number of ways you can export data to excel
- The first example appends these new calculations onto a copy of the "my_input" dataframe. This means you can export a spreadsheet that looks exactly like your input spreadsheet, but with the values for the new calculations

In [40]:
# First make a copy of the dataframe so you don't overwrite the original
my_input_c=my_input.copy()
# To make a new column, you specify my_input_c['new column name']= value
my_input_c['T_Helz1987']=T_Helz1987
# You can add as many new columns as you wish
my_input_c['Teq15_4H2O']=Teq15_4H2O
# You can give your columns in the new dataframe any name you wish...
my_input_c['Temp eq 15 4 wt% H2O']=Teq15_4H2O
# Now, you ue the "to_excel" panda function to save this new appended dataframe to excel. The name of the dataframe you've been appended to goes on the let, and the name of the
# file you want to make goes bewteen the ' ' signs
my_input_c.to_excel('Thermometry_out1.xlsx')